# CSV mit Python verarbeiten (bokeh)

Dieses Notebook möchte eine alternative Library vorstellen, um Datenvisualisierungen mit Python zu machen mit interaktiven Elementen.
Wir verwenden dazu [bokeh](https://docs.bokeh.org/).

Dieses Notebook basiert auf einem [Tutorial von RealPython](https://realpython.com/python-data-visualization-bokeh/).

Als Beispiel-Datensatz nehmen wir den Datensatz [«Aufenthaltsart der ausländischen Bevölkerung nach Nationalität und Geschlecht seit 1993»](https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_nationalitaet_aufenthaltsart_geschl_od3371).
Dieser Datensatz beinhaltet keine Einzeldaten, sondern die aggregierte Anzahl an Personen nach Nationalität und Geschlecht. 

In [101]:
import math
import requests
import pandas as pd
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show, reset_output
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel, Slider
from bokeh.models.callbacks import CustomJS

In [102]:
result = requests.get(
    'https://data.stadt-zuerich.ch/api/3/action/package_show?id=bev_bestand_jahr_nationalitaet_aufenthaltsart_geschl_od3371'
)
dataset = result.json()['result']
stay_url = dataset['resources'][0]['url']
df = pd.read_csv(stay_url)
df

,StichtagDatJahr,AufArt2CdAgg,AufArt2LangAgg,SexCd,SexLang,NationHistCd,NationHistLang,RegionCd,RegionLang,KontinentCd,KontinentLang,AnzBestWir
0,1993,1,Niedergelassene (C),1,männlich,8201,Albanien,17,Südosteuropa,1,Europa,1
1,1993,1,Niedergelassene (C),1,männlich,8204,Belgien,14,Westeuropa,1,Europa,34
2,1993,1,Niedergelassene (C),1,männlich,8205,Bulgarien,17,Südosteuropa,1,Europa,15
3,1993,1,Niedergelassene (C),1,männlich,8206,Dänemark,11,Nordeuropa,1,Europa,46
4,1993,1,Niedergelassene (C),1,männlich,8207,Deutschland,13,Zentraleuropa,1,Europa,2858
5,1993,1,Niedergelassene (C),1,männlich,8211,Finnland,11,Nordeuropa,1,Europa,29
6,1993,1,Niedergelassene (C),1,männlich,8212,Frankreich,14,Westeuropa,1,Europa,292
7,1993,1,Niedergelassene (C),1,männlich,8214,Griechenland,16,Südeuropa,1,Europa,1018
8,1993,1,Niedergelassene (C),1,männlich,8215,Grossbritannien,14,Westeuropa,1,Europa,343
9,1993,1,Niedergelassene (C),1,männlich,8215,Grossbritannien,42,Ostasien,4,Asien,6


In [113]:
# Prepare the data
year_data = {}
for year in range(1993, 2019):
    year_data[year] = df[df.StichtagDatJahr == year].groupby(['RegionLang']).size().reset_index(name='Anzahl').sort_values(by=['Anzahl'], ascending=False).reset_index(drop=True)

year_data[2018]

,RegionLang,Anzahl
0,Westafrika,74
1,Südosteuropa,59
2,Westasien,52
3,Südostasien,48
4,Zentraleuropa,42
5,Osteuropa,39
6,Südasien,37
7,Karibik,37
8,Westeuropa,36
9,Nordafrika,36


In [114]:
regions = year_data[2018]['RegionLang'].values.tolist()
regions

['Westafrika',
 'Südosteuropa',
 'Westasien',
 'Südostasien',
 'Zentraleuropa',
 'Osteuropa',
 'Südasien',
 'Karibik',
 'Westeuropa',
 'Nordafrika',
 'Ostafrika',
 'Zentralamerika',
 'Nordeuropa',
 'Südeuropa',
 'Zentralafrika',
 'Ostasien',
 'Mittleres Südamerika',
 'Zentralasien',
 'Südwestasien',
 'Südliches Südamerika',
 'Südostafrika',
 'Südliches Ozeanien',
 'Südafrika',
 'Nördliches Südamerika',
 'Nordamerika',
 'Südwesteuropa',
 'Unzuteilbar',
 'Südwestafrika',
 'Nördliches Asien',
 'Mittleres Ozeanien']

In [105]:
reset_output()
# Determine where the visualization will be rendered
#output_file('filename.html')  # Render to static HTML, or 
output_notebook()  # Render inline in a Jupyter Notebook

# Use reset_output() between subsequent show() calls, as needed


Loading BokehJS ...

In [116]:
data_2018_cds = ColumnDataSource(year_data[2018])

# Set up the figure(s)
p = figure(plot_width=1400, plot_height=400, x_range=regions)
p.xaxis.major_label_orientation = (math.pi/2) * 0.5

p.vbar(x='RegionLang',
       source=data_2018_cds,
       width=0.5,
       top='Anzahl',
       color="navy")

# Organize the layout

# Preview and save 
show(p)  # See what I made, and save if I like it

In [123]:
year_cds = {k: ColumnDataSource(v) for (k,v) in year_data.items()}
dummy = ColumnDataSource(data_2018)

p = figure(plot_width=950, plot_height=600, x_range=regions)
p.xaxis.major_label_orientation = (math.pi/2) * 0.5

p.vbar(x='RegionLang',
       source=dummy,
       width=0.5,
       top='Anzahl',
       color="navy")

callback = CustomJS(args={'source': dummy,
                          'years': year_cds,
                         }, code="""
                         
    
    var year = cb_obj.value
    var data = source.data;
    var new_data = years[year].data;
    
    //clears the dummy datasource
    for (var e in data) {
        delete data[e];
    }
    
    data['Anzahl'] = new_data['Anzahl']
    data['RegionLang'] = new_data['RegionLang'];
    data['index'] = new_data['index'];
    
    console.log(data)
    
    source.change.emit();
""")

slider = Slider(start=1993, end=2018, value=2018, step=1, title="Jahr")
slider.js_on_change('value', callback)

layout = column(slider, p)

show(layout)